In [1]:
import pandas as pd
import re

In [2]:
data_url = "https://raw.githubusercontent.com/apogiatzis/race-bar-chart-unemployment/master/data/unemployment_per_economic_activity_monthly.csv"
df = pd.read_csv(data_url)
df.head()

,NACE 2 CODE,Activity Code,Economic Activity,Year,Month,Anastoles-Metapoiisi,Anastoles-Touristiki,Termatismoi-Touristiki,Termatismoi-Alloi,Total
0,A,1,"Γεωργία, δασοκομία και αλιεία",2013,Ιανουάριος,0,0,0,159,159
1,B,2,Ορυχεία και λατομεία,2013,Ιανουάριος,0,0,0,62,62
2,C,3,Μεταποίηση,2013,Ιανουάριος,141,0,0,2690,2831
3,D,4,"Παροχή ηλεκτρικού ρεύματος, φυσικού αερίου, ατ...",2013,Ιανουάριος,0,0,0,15,15
4,E,5,"Παροχή νερού, επεξεργασία λυμάτων, διαχείριση ...",2013,Ιανουάριος,0,0,0,31,31


In [3]:
## Month Translation table 
## The prefix is to maintain sorted columns
month_gr2en = {'Ιανουάριος': '_01 Jan', 'Φεβρουάριος': '_02 Feb', 'Μάρτιος': '_03 Mar',
               'Απρίλιος': '_04 Apr', 'Μάιος':'_05 May', 'Ιούνιος':'_06 Jun',
               'Ιούλιος': '_07 Jul', 'Αύγουστος':'_08 Aug', 'Σεπτέμβριος': '_09 Sep',
               'Οκτώβριος':'_10 Oct', 'Νοέμβριος':'_11 Nov',
               'Δεκέμβριος': '_12 Dec'}

In [4]:
## Group by Economic Activity just to get the unique EA categories.
## Could be done with .unique() method but this makes sure there are
## not dangling categories.
df_grouped = df.groupby('Economic Activity')

## Group by Year and Month to group up the data 
## of the new dataframe per column.
df_year_groups = df.groupby(['Year', 'Month'])

In [5]:
EA_TOTAL_COLUMNS = [2, 9] # Indices of Economic Activity and Total columns

formatted_df = pd.DataFrame(index=df_grouped.groups.keys()) # Empty DF, only indices

## Create a new dataframe with the total unemployment per economic activity
## and concat with the previous one
for k,v in df_year_groups.groups.items():
  column_label = str(k[0]) + ' ' + month_gr2en[k[1]]
  aggregated = df.iloc[v, EA_TOTAL_COLUMNS].set_index('Economic Activity')
  aggregated.rename(columns={'Total': column_label}, inplace=True)
  formatted_df = pd.concat([formatted_df, aggregated], axis=1, join_axes=[formatted_df.index])  

## Because of groupby, columns are not sorted. Sort month colums
sorted_columns = list(formatted_df.columns.sort_values())

## Add category field + the sorted columns
formatted_df['category'] = range(len(df_grouped.groups.keys()))
formatted_df = formatted_df[['category'] + sorted_columns]

## Remove the prefix used for ordering from the month oclumns
formatted_df.columns = list(map(lambda col: re.sub(r"_[0-9]*[ \t]+","", col),
                               formatted_df.columns))

## Set Index
formatted_df.index.name = 'Economic Activity'

## Remove Unwanted Eaconomic Activity categories
formatted_df = formatted_df.drop(['Μη δηλωμένη οικονομική δραστηριότητα', 'Σύνολο'])

## Save to csv
formatted_df.to_csv('data.csv', encoding='utf-8-sig')

formatted_df.head()

TypeError: ignored